In [7]:
from sklearn.cluster import AgglomerativeClustering
from sklearn import datasets
iris = datasets.load_iris()
clustering = AgglomerativeClustering()
clustering.fit(iris.data)
clustering.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [2]:
clustering.fit_predict(iris.data)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [8]:
type(clustering.labels_)

numpy.ndarray

In [204]:
from sklearn.base import BaseEstimator, ClusterMixin
from sklearn.metrics import pairwise_distances
from scipy.spatial import distance
from operator import add
import numpy as np

class MyAgglomerativeClustering(BaseEstimator, ClusterMixin):
    def __init__(self, n_cluster=3, linkage="single"):
        assert n_cluster > 1
        self.n_cluster = n_cluster
        self.linkage = linkage
    def fit(self, X, y=None):
        distance_matrix = pairwise_distances(X)
        self.labels_ = np.zeros(X.shape[0])
        
        #initiate cluster groups
        cluster = []
        for i in range(X.shape[0]):
            init_cluster = [i]
            cluster.append(init_cluster)
        # merges cluster until n_cluster achieved
        while distance_matrix.shape[0] > self.n_cluster:
            # search minimum dissimilarity to merge once
            print(distance_matrix.shape[0])
            print(distance_matrix)
            min_i, min_j = self.get_minimum_idx(distance_matrix)
            print (min_i, min_j)
            # merge cluster members
            for merger in cluster[min_j]:
                cluster[min_i].append(merger)
            del cluster[min_j]
            print(cluster)
            # renew distance matrix
            distance_matrix = np.delete(distance_matrix, min_j, 0)
            distance_matrix = np.delete(distance_matrix, min_j, 1)
            # recompute distance matrix
            if self.linkage=="single":
                minDist = None
                for i in range (0,distance_matrix.shape[0]):
                    if (i==min_i):
                        continue
                    else:
                        for clusterItem0 in cluster[min_i]:
                            for clusterItem1 in cluster[i]:
                                if (minDist == None):
                                    minDist = distance.euclidean(X[clusterItem0],X[clusterItem1])
                                elif distance.euclidean(clusterItem0,clusterItem1) < minDist:
                                    minDist = distance.euclidean(X[clusterItem0],X[clusterItem1])
                    distance_matrix[min_i,i] = minDist;
                    distance_matrix[i,min_i] = minDist;
            elif self.linkage=="complete":
                maxDist = None
                for i in range (0,distance_matrix.shape[0]):
                    if (i==min_i):
                        continue
                    else:
                        for clusterItem0 in cluster[min_i]:
                            for clusterItem1 in cluster[i]:
                                if (maxDist == None):
                                    maxDist = distance.euclidean(X[clusterItem0],X[clusterItem1])
                                elif distance.euclidean(clusterItem0,clusterItem1) > maxDist:
                                    maxDist = distance.euclidean(X[clusterItem0],X[clusterItem1])
                    distance_matrix[min_i,i] = maxDist;
                    distance_matrix[i,min_i] = maxDist;
            elif self.linkage=="average_group":
            #TBD
                for i in range (0,distance_matrix.shape[0]):
                    if (i==min_i):
                        continue
                    else:
                        mean0 = [0]*(X.shape[1])
                        for clusterItem0 in cluster[min_i]:
                            mean0 = list(map(add,mean0,X[clusterItem0]))
                        for element in mean0:
                            element = element / len(cluster[min_i])
                        mean1 = [0]*(X.shape[1])
                        for clusterItem1 in cluster[i]:
                            mean1 = list(map(add,mean1,X[clusterItem0]))
                        for element in mean1:
                            element = element / len(cluster[i])
                        dist = distance.euclidean(mean0,mean1)
                    distance_matrix[min_i,i] = dist;
                    distance_matrix[i,min_i] = dist;
            #TBD algo lain
            print(distance_matrix)
        #set labels
        i = 0
        for clust in cluster:
            label = i
            for member in clust:
                self.labels_[member] = label
            i = i+1
        print self.labels_
    def get_minimum_idx(self,m):
        assert (m.shape[0] > 1)
        min = m[1,0]
        min_i = 1;
        min_j = 0;
        for j in range(1, m.shape[0]):
            for i in range(0, j):
                if m[i,j] < min:
                    min = m[i,j]
                    min_i = i
                    min_j = j
        if min_i<min_j:
            return min_i, min_j
        return min_j, min_i
            
            

In [205]:
clustering = MyAgglomerativeClustering(linkage="average_group")

In [206]:
 X = np.array([[1, 2], [1, 4], [1, 0],[4, 2], [4, 4], [4, 0]])
clustering.fit(iris.data)

150
[[ 0.          0.53851648  0.50990195 ...,  4.45982062  4.65080638
   4.14004831]
 [ 0.53851648  0.          0.3        ...,  4.49888875  4.71805044
   4.15331193]
 [ 0.50990195  0.3         0.         ...,  4.66154481  4.84871117
   4.29883705]
 ..., 
 [ 4.45982062  4.49888875  4.66154481 ...,  0.          0.6164414
   0.64031242]
 [ 4.65080638  4.71805044  4.84871117 ...,  0.6164414   0.          0.76811457]
 [ 4.14004831  4.15331193  4.29883705 ...,  0.64031242  0.76811457  0.        ]]
(101, 142)
[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32], [33], [34], [35], [36], [37], [38], [39], [40], [41], [42], [43], [44], [45], [46], [47], [48], [49], [50], [51], [52], [53], [54], [55], [56], [57], [58], [59], [60], [61], [62], [63], [64], [65], [66], [67], [68], [69], [70], [71], [72], [73], [74], [75], [76], [77], [78], [79], [80], [81], [82], [

[[  0.          11.58015544  19.41210962 ...,  19.41210962  19.41210962
   19.41210962]
 [ 11.58015544   0.           5.91692488 ...,   5.91692488   5.91692488
    5.91692488]
 [ 19.41210962   5.91692488   0.         ...,   4.66154481   4.84871117
    4.29883705]
 ..., 
 [ 19.41210962   5.91692488   4.66154481 ...,   0.           0.6164414
    0.64031242]
 [ 19.41210962   5.91692488   4.84871117 ...,   0.6164414    0.
    0.76811457]
 [ 19.41210962   5.91692488   4.29883705 ...,   0.64031242   0.76811457
    0.        ]]
133
[[  0.          11.58015544  19.41210962 ...,  19.41210962  19.41210962
   19.41210962]
 [ 11.58015544   0.           5.91692488 ...,   5.91692488   5.91692488
    5.91692488]
 [ 19.41210962   5.91692488   0.         ...,   4.66154481   4.84871117
    4.29883705]
 ..., 
 [ 19.41210962   5.91692488   4.66154481 ...,   0.           0.6164414
    0.64031242]
 [ 19.41210962   5.91692488   4.84871117 ...,   0.6164414    0.
    0.76811457]
 [ 19.41210962   5.91692488   4

[[  0.           6.33245608   0.70710678 ...,  19.41210962  19.41210962
   19.41210962]
 [  6.33245608   0.           0.70710678 ...,  19.22004162  19.22004162
   19.22004162]
 [  0.70710678   0.70710678   0.         ...,  17.52170083  17.52170083
   17.52170083]
 ..., 
 [ 19.41210962  19.22004162  17.52170083 ...,   0.           0.6164414
    0.64031242]
 [ 19.41210962  19.22004162  17.52170083 ...,   0.6164414    0.
    0.76811457]
 [ 19.41210962  19.22004162  17.52170083 ...,   0.64031242   0.76811457
    0.        ]]
110
[[  0.           6.33245608   0.70710678 ...,  19.41210962  19.41210962
   19.41210962]
 [  6.33245608   0.           0.70710678 ...,  19.22004162  19.22004162
   19.22004162]
 [  0.70710678   0.70710678   0.         ...,  17.52170083  17.52170083
   17.52170083]
 ..., 
 [ 19.41210962  19.22004162  17.52170083 ...,   0.           0.6164414
    0.64031242]
 [ 19.41210962  19.22004162  17.52170083 ...,   0.6164414    0.
    0.76811457]
 [ 19.41210962  19.22004162  17

[[  0.           6.33245608   0.70710678 ...,   1.1045361   19.41210962
   19.41210962]
 [  6.33245608   0.           0.70710678 ...,   1.1045361   19.22004162
   19.22004162]
 [  0.70710678   0.70710678   0.         ...,   1.1045361   17.52170083
   17.52170083]
 ..., 
 [  1.1045361    1.1045361    1.1045361  ...,   0.          28.15350777
   28.15350777]
 [ 19.41210962  19.22004162  17.52170083 ...,  28.15350777   0.
    0.65574385]
 [ 19.41210962  19.22004162  17.52170083 ...,  28.15350777   0.65574385
    0.        ]]
83
[[  0.           6.33245608   0.70710678 ...,   1.1045361   19.41210962
   19.41210962]
 [  6.33245608   0.           0.70710678 ...,   1.1045361   19.22004162
   19.22004162]
 [  0.70710678   0.70710678   0.         ...,   1.1045361   17.52170083
   17.52170083]
 ..., 
 [  1.1045361    1.1045361    1.1045361  ...,   0.          28.15350777
   28.15350777]
 [ 19.41210962  19.22004162  17.52170083 ...,  28.15350777   0.
    0.65574385]
 [ 19.41210962  19.22004162  1

48
[[  0.          24.59878046  24.59878046 ...,  45.5779552   45.5779552
   24.59878046]
 [ 24.59878046   0.           0.70710678 ...,  19.22004162  19.22004162
    1.1045361 ]
 [ 24.59878046   0.70710678   0.         ...,  17.52170083  17.52170083
    1.1045361 ]
 ..., 
 [ 45.5779552   19.22004162  17.52170083 ...,   0.           2.14709106
   28.15350777]
 [ 45.5779552   19.22004162  17.52170083 ...,   2.14709106   0.
   28.15350777]
 [ 24.59878046   1.1045361    1.1045361  ...,  28.15350777  28.15350777
    0.        ]]
(26, 38)
[[0, 17, 10, 48, 9, 34, 101, 142], [1, 12, 95, 96], [2, 47, 29, 30], [3, 45, 107, 130], [4, 40, 94, 99], [5, 18, 83, 133], [6, 11, 20, 31], [7, 39, 128, 132], [8, 38, 19, 21], [13, 42, 55, 90], [14, 15, 103, 134], [16, 36, 64, 79], [22], [23, 26, 53, 89], [24, 43, 59, 88], [25, 37, 66, 84], [27, 28, 80, 81], [32, 33, 76, 86], [35, 49, 110, 147], [41], [44, 46, 125, 129], [50, 52, 51, 56], [54, 58, 141, 145], [57, 93, 116, 137], [60], [61, 78, 113, 121], [62

[[   0.           52.75926459   52.75926459   52.75926459   52.75926459
    52.75926459   52.75926459   76.74939739   85.86792183   85.86792183
    85.86792183   86.91961804   85.86792183   85.86792183   52.75926459
    85.86792183   85.86792183   85.86792183   85.86792183   85.86792183
    85.86792183   85.86792183   85.86792183   85.86792183   85.86792183
   102.65914475]
 [  52.75926459    0.            9.95540054   15.68119893    9.12578764
     5.2744668    16.26222617    9.84225584   26.45108693   26.45108693
    43.37476225   14.3881896    26.45108693   26.45108693    5.36842621
    26.45108693   26.45108693   26.45108693   41.9181345    26.45108693
    26.45108693   26.45108693   26.45108693   40.26251358   26.45108693
    66.02916022]
 [  52.75926459    9.95540054    0.            9.95540054    9.12578764
     5.2744668    16.26222617    9.84225584   30.14083609   30.14083609
    30.14083609   14.3881896    30.14083609   30.14083609    5.36842621
    30.14083609   30.14083609 

[[   0.           29.94428159   17.78285691   22.6792857    20.45947213
    30.6245       26.31919452   41.06896639   85.86792183  108.72120308
   101.20093873  102.65914475]
 [  29.94428159    0.           17.78285691   22.6792857    20.45947213
    30.6245       26.31919452   41.06896639   80.64304062  108.72120308
   101.20093873   97.69431918]
 [  17.78285691   17.78285691    0.           22.6792857    20.45947213
    30.6245       26.31919452   41.06896639   83.63581769  108.72120308
   101.20093873   98.62854556]
 [  22.6792857    22.6792857    22.6792857     0.           20.45947213
    30.6245       26.31919452   41.06896639   98.86976282   59.98091363
    59.98091363  118.4185374 ]
 [  20.45947213   20.45947213   20.45947213   20.45947213    0.           30.6245
    26.31919452   41.06896639   90.40459059   56.59143045   56.59143045
   107.48390577]
 [  30.6245       30.6245       30.6245       30.6245       30.6245        0.
    26.31919452   41.06896639   82.52236061   50.14

In [186]:
[1,3,4] + [5,10,-2]


[1, 3, 4, 5, 10, -2]

In [200]:
X.shape

(6L, 2L)